# 20160923 IMSE Thoughts...

Try to use Jupyter/Python Kernel/ to manage my worktime/sparetime

尝试使用Jupyter来管理我的加班/请假信息，管理我的历史，给出今年，近月 加班时间/请假时间

定好需求/设计格式/通用/设计函数

> 20160921 3.5hours STA_JOB设计

> 20160923 2hours SDM_JOB设计

> 20160926 1.5hours SDM_JOB定长

---
需求：
0. 输入就是一行一行的 加班/请假 记录，注意格式应该简单易懂易用;
1. 能够打印当前月的 加班/请假 信息;
2. 能够显示 （当年|累计） 加班时间/请假时间/可用时间; 

---
改进:
0. 需要美化输出,格式化输出.


In [4]:
## -*- coding : utf-8 -*-

import datetime

records = """
20170413 1.5hours 客户数据查错修正
20170410 1hours 短信程序改版上线
20170311 5hours 绩-提供指标啊
20170310 2hours 绩-指标们啊啊啊
20170307 2hours 小薇-导表
20170306 1hours 绩-数据验证
20170201 2hours -两份查询
20170119 2hours 信-开发测试
"""

#print( 'This year is ' + str(run_timestamp.year ) )
#print( 'This month is ' + str(run_timestamp.month ) )
#print( records )

# { YYYY , { MM , ( DD, HOURS, COMMENTS ) }
# { key  , { key , ( xx, xx, xx), (xx, xx, xx) ) }


class OvertimeAndHoliday:
    
    
    def __init__( self, org_records ):
        
        # A memory dict to store info
        self.database = dict()
        
        splited_records = org_records.strip('\n').split('\n')
        
        for r in splited_records:
            #print( r )
            elements = r.split(' ')
            th_date = elements[0]
            th_hours = elements[1]
            th_comme = elements[2]
            
            # YYYY MM DD; They are integers, not a string...
            this_year = int(th_date[0:4])
            this_month = int(th_date[4:6])
            this_day = int(th_date[6:])
            
            # this_hours > 0 Overtime; This_hours<0 holiday...
            this_hours = float(th_hours[:-5])
            
            # get the value out
            inner_dict = self.database.get( this_year, dict() )
            inner_lst = inner_dict.get( this_month, list() )
            inner_lst.append( (this_day, this_hours, th_comme) )
            
            # put the value back
            inner_dict[this_month] = inner_lst
            self.database[this_year] = inner_dict
            
    def print_year_statistics( self, YYYY ):
        # YYYY is a Integer 打印指定年的加班/休假/剩余时间 统计信息
        given_year = YYYY
        
        months = range(1,13)
        
        year_total_ot_hours = 0.0
        year_total_hl_hours = 0.0
        year_remain_ot_hours = 0.0
        
        for m in months:
            th_mon_tuple = self.print_month_record( given_year, m, PRINT='N' )
            
            year_total_ot_hours += th_mon_tuple[0]
            year_total_hl_hours += th_mon_tuple[1]
            year_remain_ot_hours += th_mon_tuple[2]
            
        print( "\n====================================================\n" )
        print( " 在 " + str(given_year) + " 年度 :\n" )
        print( " 总共加班时间 : " + str(year_total_ot_hours) + " 小时\n" )
        print( " 总共休假时间 : " + str(year_total_hl_hours) + " 小时\n" )
        print( " 可用休假时间 : " + str(year_remain_ot_hours) + " 小时\n" )
        print( "====================================================\n" )
    
    def print_month_record( self, YYYY, MM, PRINT='Y' ):
        # YYYY, MM are Integers, 统计/打印指定 年 月 的加班/休假/剩余时间
        
        given_year = YYYY
        given_month = MM
        
        total_ot_hours = 0.0
        total_hl_hours = 0.0
        remain_ot_hours = 0.0
        
        #out_string = ""
        out_string = "\n正在打印 " + str(given_year) + "-" + str(given_month) + " 的加班/休假信息...\n\n"
        
        # always know what you are handling
        year_dict = self.database.get( given_year, dict() )
        month_list = year_dict.get( given_month, list() )
        
        for rec in month_list:
            day = rec[0]
            hours = rec[1]
            comment = rec[2]
            
            if hours > 0 :
                total_ot_hours += hours
            else:
                total_hl_hours += abs(hours)
            
            out_string += "日期:\t" + str(given_year) + "-" + str(given_month) + "-" + str(day) + " :   " \
                  + str(hours) + " 小时, 原因 :   " + comment + "\n"
            
        remain_ot_hours = total_ot_hours - total_hl_hours
        
        out_string += "\n====================================================\n"
        out_string += " In " + str(given_year) + "-" + str(given_month) + " :\n"
        out_string += " 总共加班时间 : " + str(total_ot_hours) + " 小时\n"
        out_string += " 总共休假时间 : " + str(total_hl_hours) + " 小时\n"
        out_string += " 可用休假时间 : " + str(remain_ot_hours) + " 小时\n"
        out_string += "====================================================\n"
        
        if 'Y' == PRINT :
            print out_string
            
        # Return This Month Tuple ( total_ot_hours, total_hl_hours, remain_ot_hours )
        return ( total_ot_hours, total_hl_hours, remain_ot_hours )
        
        
OAH_obj = OvertimeAndHoliday(records)

# Print Current Year-Month
run_timestamp = datetime.datetime.now()
run_year = run_timestamp.year
run_month = run_timestamp.month

mon_tuple = OAH_obj.print_month_record( run_year, run_month )

# Print Previous Year_Month
pre_year = run_year
pre_month = run_month - 1
if 0 == pre_month :
    pre_month = 12
    pre_year -= 1

#mon_tuple = OAH_obj.print_month_record( pre_year, pre_month )

OAH_obj.print_year_statistics( run_year )



正在打印 2017-4 的加班/休假信息...

日期:	2017-4-10 :   1.0 小时, 原因 :   短信程序改版上线

 In 2017-4 :
 总共加班时间 : 1.0 小时
 总共休假时间 : 0.0 小时
 可用休假时间 : 1.0 小时



 在 2017 年度 :

 总共加班时间 : 15.0 小时

 总共休假时间 : 0.0 小时

 可用休假时间 : 15.0 小时


